### Sources Used  
[Traffic sign dataset](https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification)
[Example notebook on traffic sign dataset](https://www.kaggle.com/code/moh3we5/traffic-sign-dataset-resnet-classification)
[Tensor flow transfer learning documentation](https://www.tensorflow.org/guide/keras/transfer_learning)

### Introduction
A Traffic sign dataset will be used, to create a CNN capable of recognizing common traffic signs, these datasets are often used as a component of self driving AI. While most of those implementations tend to use multiple models to narrow down the kind of sign, thereby improving possible classification even with small deviations between similar signs.  
In this notebook a single large model will be trained however.

The dataset used contains 58 classes, covering most common signs, however some classes are labeled as Unkown[n].
For every class ~120 images are present.

In this notebook, transfer learning will be leveraged to obtain a pretrained base model. By using transfer learning, the training process can be greatly reduced and will allow us to utilize layers that have been trained on much larger datasets than the one used here.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Load in the traffic sign dataset

In [ ]:
dataset_directory = "Dataset/traffic_Data/DATA/"

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(dataset_directory, labels='inferred', seed=123)

First, obtain a pre trained model, this can be done via the tensorflow keras API.  
Here we specify a model that has its weights trained on the ImageNet dataset, the expected image input size and that the classification top layer should be excluded

In [2]:
pretrained_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    input_shape=(150, 150, 3), # Input shape of our dataset images
    include_top=False)  # Exclude the classifier layer, as our own will be used

83683744/83683744 [==============================] - 7s 0us/step


Block the pretrained layers/base-model from having its weights changed during training

In [3]:
pretrained_model.trainable = False

Now, lets create our own model

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = pretrained_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

Lets train this new model

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(new_dataset, epochs=20, callbacks=..., validation_data=...)